In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd '/content/drive/MyDrive/Github/Deceptive-Review-Detection'

/content/drive/MyDrive/Github/Deceptive-Review-Detection


In [4]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-8f90b03a-6491-132c-ef39-0ff5b1e244c4)


In [5]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import torch
import torch.nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

from transformers import BertTokenizer, BertModel, BertConfig, AdamW, get_linear_schedule_with_warmup
import sys

In [8]:
import data_prep
from BERT_Classification import *

In [9]:
# df = data_prep.import_chi_review()
df = pd.read_csv("Chicago_Hotel_Review/Chicago_Hotel_Reviews.csv", index_col=0)
df = df.drop('Rating', axis=1)
df.head()

,Label,Ori_Review
0,1,Affinia Chicago is one of the worst hotels I h...
1,1,I recently stayed at the Affina Chicago hotel ...
2,1,I stayed at the Affina Chicago for my annivers...
3,1,If you are looking for a high end hotel on the...
4,1,I just returned from a long weekend in Chicago...


In [10]:
X = df['Ori_Review']
y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=0)
X_train = X_train.to_list()
X_test = X_test.to_list()
X_val = X_val.to_list()
y_train = y_train.to_list()
y_test = y_test.to_list()
y_val = y_val.to_list()

In [11]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Device used: {}.".format(device))

Device used: cuda.


In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
seq_length = 300
batch_size = 16

In [14]:
train_dataset = dataset(tokenizer, X_train, y_train, max_seq_length=seq_length)
val_dataset = dataset(tokenizer, X_val, y_val, max_seq_length=seq_length)
test_dataset = dataset(tokenizer, X_test, y_test, max_seq_length=seq_length)

In [15]:
train_data_loader = DataLoader(dataset=train_dataset, 
                         batch_size=batch_size, 
                         shuffle=True)
val_data_loader = DataLoader(dataset=val_dataset, 
                         batch_size=batch_size, 
                         shuffle=True)
test_data_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=True)

In [16]:
model = BertWithClassifier(bert, seq_length, nn.CrossEntropyLoss).to(device)

In [17]:
lrs = [5e-5, 3e-5, 2e-5]
batch_sizes = [16, 32]
num_epochs = [2, 3, 4]

In [18]:
train_losses, val_losses = train(model=model, 
                                data_loader=train_data_loader, 
                                val_data_loader=val_data_loader, 
                                optimizer=AdamW, 
                                learning_rate=3e-5,
                                num_epochs=30,
                                device=device,
                                scheduler=get_linear_schedule_with_warmup,
                                batch_size=batch_size,
                                data_size=len(train_dataset))

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


training:   0%|          | 0/30 [00:00<?, ?epoch/s]

epoch 1:   0%|          | 0/81 [00:00<?, ?batch/s]

tensor(0.6111, device='cuda:0')


epoch 2:   0%|          | 0/81 [00:00<?, ?batch/s]

...

tensor(0.8194, device='cuda:0')


epoch 29:   0%|          | 0/81 [00:00<?, ?batch/s]

tensor(0.8056, device='cuda:0')


epoch 30:   0%|          | 0/81 [00:00<?, ?batch/s]

tensor(0.8194, device='cuda:0')


In [19]:
train_losses, val_losses

([tensor([0.6999], device='cuda:0', grad_fn=<DivBackward0>),
  tensor([0.6957], device='cuda:0', grad_fn=<DivBackward0>),
  tensor([0.6887], device='cuda:0', grad_fn=<DivBackward0>),
  tensor([0.6974], device='cuda:0', grad_fn=<DivBackward0>),
  tensor([0.6864], device='cuda:0', grad_fn=<DivBackward0>),
  tensor([0.6990], device='cuda:0', grad_fn=<DivBackward0>),
  tensor([0.6928], device='cuda:0', grad_fn=<DivBackward0>),
  tensor([0.6901], device='cuda:0', grad_fn=<DivBackward0>),
  tensor([0.6985], device='cuda:0', grad_fn=<DivBackward0>),
  tensor([0.6965], device='cuda:0', grad_fn=<DivBackward0>),
  tensor([0.6929], device='cuda:0', grad_fn=<DivBackward0>),
  tensor([0.6917], device='cuda:0', grad_fn=<DivBackward0>),
  tensor([0.6922], device='cuda:0', grad_fn=<DivBackward0>),
  tensor([0.6953], device='cuda:0', grad_fn=<DivBackward0>),
  tensor([0.6920], device='cuda:0', grad_fn=<DivBackward0>),
  tensor([0.6802], device='cuda:0', grad_fn=<DivBackward0>),
  tensor([0.6336], devic

In [20]:
# test_preds, test_loss, test_acc = validate(model, test_data_loader, device)

In [21]:
# test_acc